In [1]:
from openforcefield.typing.engines.smirnoff import ForceField
from openforcefield.topology.molecule import Molecule
from openforcefield.topology.topology import Topology

from system.typing.smirnoff import build_smirnoff_map, build_smirnoff_collection
from system.system import System

In [2]:
# Load in a mainline OpenFF forcefield and construct a minimal ethanol topology
parsley = ForceField('openff-1.0.0.offxml')

top =  Topology.from_molecules(Molecule.from_smiles('CCO'))

In [3]:
# Generate a (partial) mapping of topology slots and SMIRNOFF SMIRKS
smirks_map = build_smirnoff_map(forcefield=parsley, topology=top)

smirks_map

{'vdW': {(0,): '[#6X4:1]',
  (1,): '[#6X4:1]',
  (2,): '[#8X2H1+0:1]',
  (3,): '[#1:1]-[#6X4]',
  (4,): '[#1:1]-[#6X4]',
  (5,): '[#1:1]-[#6X4]',
  (6,): '[#1:1]-[#6X4]-[#7,#8,#9,#16,#17,#35]',
  (7,): '[#1:1]-[#6X4]-[#7,#8,#9,#16,#17,#35]',
  (8,): '[#1:1]-[#8]'},
 'Bonds': {(0, 1): '[#6X4:1]-[#6X4:2]',
  (0, 3): '[#6X4:1]-[#1:2]',
  (0, 4): '[#6X4:1]-[#1:2]',
  (0, 5): '[#6X4:1]-[#1:2]',
  (1, 2): '[#6:1]-[#8:2]',
  (1, 6): '[#6X4:1]-[#1:2]',
  (1, 7): '[#6X4:1]-[#1:2]',
  (2, 8): '[#8:1]-[#1:2]'}}

In [4]:
# This is not efficient at the moment, will take a minute or so to run
smirnoff_collection = build_smirnoff_collection(forcefield=parsley)

In [5]:
# This is like the smirnoff_collection in the other notebook, but encompasses more parameters
# (all vdW and bond parameters in the force field). You can inspect, for example, the list
# of all SMIRKS patterns of bonds
smirnoff_collection.handlers['Bonds'].potentials.keys()

dict_keys(['[#6X4:1]-[#6X4:2]', '[#6X4:1]-[#6X3:2]', '[#6X4:1]-[#6X3:2]=[#8X1+0]', '[#6X3:1]-[#6X3:2]', '[#6X3:1]:[#6X3:2]', '[#6X3:1]=[#6X3:2]', '[#6:1]-[#7:2]', '[#6X3:1]-[#7X3:2]', '[#6X4:1]-[#7X3:2]-[#6X3]=[#8X1+0]', '[#6X3:1](=[#8X1+0])-[#7X3:2]', '[#6X3:1]-[#7X2:2]', '[#6X3:1]:[#7X2,#7X3+1:2]', '[#6X3:1]=[#7X2,#7X3+1:2]', '[#6:1]-[#8:2]', '[#6X4:1]-[#8X2H0:2]', '[#6X3:1]-[#8X2:2]', '[#6X3:1]-[#8X2H1:2]', '[#6X3a:1]-[#8X2H0:2]', '[#6X3:1](=[#8X1])-[#8X2H0:2]', '[#6:1]=[#8X1+0,#8X2+1:2]', '[#6X3:1](~[#8X1])~[#8X1:2]', '[#6X3:1]~[#8X2+1:2]~[#6X3]', '[#6X2:1]-[#6:2]', '[#6X2:1]-[#6X4:2]', '[#6X2:1]=[#6X3:2]', '[#6:1]#[#7:2]', '[#6X2:1]#[#6X2:2]', '[#6X2:1]-[#8X2:2]', '[#6X2:1]-[#7:2]', '[#6X2:1]=[#7:2]', '[#16:1]=[#6:2]', '[#6X2:1]=[#16:2]', '[#7:1]-[#7:2]', '[#7X3:1]-[#7X2:2]', '[#7X2:1]-[#7X2:2]', '[#7:1]:[#7:2]', '[#7:1]=[#7:2]', '[#7:1]#[#7:2]', '[#7:1]-[#8X2:2]', '[#7:1]~[#8X1:2]', '[#8X2:1]-[#8X2:2]', '[#16:1]-[#6:2]', '[#16:1]-[#1:2]', '[#16:1]-[#16:2]', '[#16:1]-[#9:2]', '[#1

In [6]:
# Or look up a specific parameter within a particlar bond
smirnoff_collection.handlers['Bonds'].potentials['[#6X3:1](~[#8X1])~[#8X1:2]'].parameters['k']

1226.156681261 <Unit('kilocalorie / angstrom ** 2 / mole')>

In [7]:
# Can build up a system object using the data we have
ethanol_system = System(
    potential_collection=smirnoff_collection,
    topology=top,
)

In [8]:
# And using the SMIRKS map (which should be merged into the System at some point), we can
# find the SMIRKS that was assigned to an arbitrary bond, say index 4
bond_key = tuple([a.topology_atom_index for a in ethanol_system.topology.bond(4).atoms])
smirks_map['Bonds'][bond_key]

'[#6X4:1]-[#1:2]'

In [9]:
# And then using that SMIRKS as a "key", we can look up its associated potential
ethanol_system.potential_collection['Bonds'].potentials[smirks_map['Bonds'][bond_key]]

ParametrizedAnalyticalPotential(name='b83', smirks='[#6X4:1]-[#1:2]', expression='1/2*k*(length-length_0)**2', independent_variables={'length_0'}, parameters={'k': <Quantity(758.0931772913, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.092888378383, 'angstrom')>})